In [73]:
#Import functions to use for this analysis
import pandas as pd
import numpy as np
import time
import datetime as dt

In [7]:
#Import csv
df_load = pd.read_csv('~/Documents/swimdata.csv')

In [29]:
#Look at first rows of dataframe similar to R head() function 
df_load[0:6]

,Place,Bib,Name,First name,Last name,Team name,Team name 2,Distance,Category,Age,Gender,Time,Difference,% Back,% Winning,% Average
0,1,250.0,Jude Williams,Jude,Williams,Mentone CA,Non-Wetsuit,1 Mile,M1517,15.0,Male,19:15.7,-,-,100%,40.14%
1,2,2.0,Steve Allnutt,Steve,Allnutt,San Diego CA,Non-Wetsuit,1 Mile,M2529,28.0,Male,19:42.6,+0:26.9,+2.33%,97.73%,38.75%
2,3,272.0,Sandoval Martha,Sandoval,Martha,Mexicali Baja California,Non-Wetsuit,1 Mile,F1824,19.0,Female,20:02.3,+0:46.6,+4.03%,96.12%,37.73%
3,4,48.0,Daniel Escalante,Daniel,Escalante,Mexicali Baja California,Non-Wetsuit,1 Mile,M1517,15.0,Male,20:29.3,+1:13.6,+6.37%,94.01%,36.33%
4,5,134.0,CLAUDIA MACIAS,CLAUDIA,MACIAS,Mexicali Baja California,Non-Wetsuit,1 Mile,F1517,17.0,Female,20:29.4,+1:13.7,+6.38%,94.01%,36.33%
5,6,23.0,EMILIO CAMARILLO,EMILIO,CAMARILLO,Chula Vista CA,Non-Wetsuit,1 Mile,M1517,16.0,Male,20:33.9,+1:18.2,+6.77%,93.66%,36.09%


In [28]:
#Look at last rows not inclusive of index
df_load[-4:-1]

,Place,Bib,Name,First name,Last name,Team name,Team name 2,Distance,Category,Age,Gender,Time,Difference,% Back,% Winning,% Average
273,-,233.0,MICHAEL SULLIVAN,MICHAEL,SULLIVAN,San Diego CA,Wetsuit,1 Mile,M2529,28.0,Male,DNS,-,-,-,-
274,-,238.0,Lori Thompson,Lori,Thompson,Trabuco Canyon CA,Wetsuit,1 Mile,F4044,40.0,Female,DNS,-,-,-,-
275,-,245.0,Mauricio Vega Kuri Garcia,Mauricio,Vega Kuri Garcia,Calexico CA,Non-Wetsuit,1 Mile,M4549,45.0,Male,DNS,-,-,-,-


In [19]:
#Generate new dataframe with only relevant columns
df = df_load[['Place','Name','Category','Gender','Time']].iloc[0:275]

In [66]:
#Count time but also get median and standard deviation
participant_count = df.groupby(['Category']).count()['Time']

In [37]:
#Using logical operator on DNS, get percent of people by category that finished race
finish_percent = df[df.Time <> 'DNS'].groupby(['Category']).count()['Time']/df.groupby(['Category']).count()['Time']

In [147]:
df['Time Array'] = np.where(df['Time'] <> 'DNS',df['Time'].str.split(':',1),'DNS')
df['Time Minutes'] = np.where(df['Time'] <> 'DNS',df['Time Array'].str[0], 'DNS')

In [148]:
#df.drop(['Time Seconds'],axis=1)
#int(df['Time Minutes'].str[0])*60 + int(df['Time Minutes'].str[1])

#time_detail = df[df.Time <> 'DNS'].groupby(['Category']).agg({'mean':np.median('Time Minutes'),'SD':np.sd('Time Minutes')})

In [67]:
cat_summary = pd.concat([finish_percent,participant_count],axis=1)
cat_summary.columns = ['Finish Rate','Participant Count'] #Add median and standard deviation of time
cat_summary

,Finish Rate,Participant Count
Category,,
F1112,1.000000,1
F1314,1.000000,14
F1517,1.000000,9
F1824,1.000000,15
F2529,0.923077,13
F3034,1.000000,11
F3539,1.000000,4
F4044,0.900000,10
F4549,0.857143,7


In [54]:
participant_locations = df_load['Team name'].iloc[0:275]

In [146]:
#Group people by coming from California, Baja California (MX), and other US states
#for i in range(0,276):
#    if 'Baja California' in participant_locations[i]:
        
        